In [1]:
print("Abc")

Abc


In [2]:
! pip install -q kaggle

^C
ERROR: Operation cancelled by user
You should consider upgrading via the '/home/alhiris/.pyenv/versions/3.9.9/bin/python3.9 -m pip install --upgrade pip' command.


In [3]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/home/alhiris/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle competitions download -c nitro-lang-processing-1

^C
Traceback (most recent call last):
  File "/home/alhiris/.pyenv/versions/3.9.9/bin/./kaggle", line 33, in <module>
    sys.exit(load_entry_point('kaggle==1.5.12', 'console_scripts', 'kaggle')())
  File "/home/alhiris/.pyenv/versions/3.9.9/bin/./kaggle", line 25, in importlib_load_entry_point
    return next(matches).load()
  File "/home/alhiris/.pyenv/versions/3.9.9/lib/python3.9/importlib/metadata.py", line 77, in load
    module = import_module(match.group('module'))
  File "/home/alhiris/.pyenv/versions/3.9.9/lib/python3.9/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 972, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_

In [7]:
! mkdir nitro-lang-processing-1

mkdir: cannot create directory ‘nitro-lang-processing-1’: File exists


In [8]:
! unzip nitro-lang-processing-1.zip -d nitro-lang-processing-1

Archive:  nitro-lang-processing-1.zip
replace nitro-lang-processing-1/random_seed_setter.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

KeyboardInterrupt: 

In [ ]:
! pip install unidecode
! pip install transformers

In [1]:
seed = 8 

from transformers import AutoTokenizer, AutoModel, pipeline, AutoModelForTokenClassification
from tqdm import tqdm

import re
from sklearn.metrics import accuracy_score
import numpy as np
np.random.seed(seed)
np.random.RandomState(seed)

import random
random.seed(seed)

import torch
import torch.nn as nn
torch.manual_seed(seed)
torch.use_deterministic_algorithms(False)

# import jax
# jax.random.PRNGKey(seed)

from unidecode import unidecode

from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

import json


MAX_LENGTH = 128
EPOCHS = 10
BATCH_SIZE = 2


In [2]:

with open("./nitro-lang-processing-1/train.json") as fin:
    raw_train_data = json.load(fin)
with open("./nitro-lang-processing-1/test.json") as fin:
    raw_test_data = json.load(fin)
with open("./nitro-lang-processing-1/tag_to_id.json") as fin:
    tag_to_id = json.load(fin)


def read_dataset(dataset, tokenizer, train=True):
    data = []
    labels = []
    max_length = 0

    for item in dataset:
        prelucrate_item = []
        for token in item['tokens']:
            prelucrate_item.append(re.sub(r"\W+", 'n', token))
        sequence = tokenizer(
            prelucrate_item,
            is_split_into_words=True,
            padding='max_length',
            truncation=True,
            max_length=MAX_LENGTH,
            return_offsets_mapping=True

        )
        sequence = {key: torch.as_tensor(value) for key, value in sequence.items()}
        data.append(sequence)

        if train:
            encoded_labels = np.ones(len(sequence["offset_mapping"]), dtype=int) * -100
            sequence_labels = item['ner_ids']
            # set only labels whose first offset position is 0 and the second is not 0
            i = 0
            for idx, offsets in enumerate(sequence["offset_mapping"]):
                if offsets[0] == 0 and offsets[1] != 0:
                    # overwrite label
                    encoded_labels[idx] = sequence_labels[i]
                    i += 1

            # max_length = max(len(sequence), max_length)
            labels.append(torch.as_tensor(encoded_labels))
    # print(max_length)
    if train:
        return data, labels

    return data

tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")

train_data, train_labels = read_dataset(raw_train_data, tokenizer=tokenizer)

test_data = read_dataset(raw_test_data, tokenizer=tokenizer, train=False)


In [3]:
X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=seed)

model = AutoModelForTokenClassification.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1", num_labels=len(tag_to_id))
# ner_model = pipeline('ner', model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at dumitrescustefan/bert-base-romanian-cased-v1 were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification we

In [4]:
test = tokenizer(
    ["testând", "de", "trei", "ori"],
    is_split_into_words=True,
    padding='max_length',
    return_offsets_mapping=True,
    truncation=False,
    max_length=4
)
print(test)
print(tokenizer.convert_ids_to_tokens([4231, 476]))
print(tokenizer.convert_tokens_to_ids(['test', '##ând']))
print(tokenizer.convert_ids_to_tokens([23570]))



{'input_ids': [2, 4231, 476, 363, 1206, 644, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 4), (4, 7), (0, 2), (0, 4), (0, 3), (0, 0)]}
['test', '##ând']
[4231, 476]
['șarpe']


In [ ]:
print(train_labels[0])

In [5]:
def pad(samples, max_length):

    return torch.tensor([
        sample[:max_length] + [0] * max(0, max_length - len(sample))
        for sample in samples
    ])

In [ ]:
# padded_train_data = pad(train_data, 563)
# padded_train_data[0]

In [ ]:
# print(padded_train_data.shape)

In [5]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        super().__init__()
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

    def __len__(self):
        return len(self.labels)

class TestDataset(Dataset):
    def __init__(self, data, labels):
        super().__init__()
        self.data = data

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)


# print(model(torch.tensor(tokenizer.encode("Testing the model", add_special_tokens=True)).unsqueeze(0)))

# print(tokenizer.encode_plus(["Convorbiri", "literare", "."]))
print(tokenizer.convert_tokens_to_ids("[PAD]"))
print(tokenizer.convert_ids_to_tokens(10))


0
&


In [6]:
train_dataset = MyDataset(X_train, y_train)
validation_dataset = MyDataset(X_val, y_val)

In [ ]:
print(len(X_train))
print(len(y_train))

In [7]:
# instantiate the DataLoaders
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
validation_dataloader = DataLoader(
    dataset=validation_dataset,
    batch_size=BATCH_SIZE
)

In [8]:
def train_epoch(model, train_dataloader, loss_crt, optimizer, device):
    model.train()
    epoch_loss = 0.0
    epoch_acc = 0.0
    num_batches = len(train_dataloader)
    predictions = []
    labels = []
    for idx, batch in tqdm(enumerate(train_dataloader)):
        batch_data, batch_labels = batch
        sequence_ids = batch_data['input_ids'].to(device, dtype=torch.long)
        sequence_masks = batch_data['attention_mask'].to(device)
        batch_labels = batch_labels.to(device)

        raw_output = model(input_ids=sequence_ids, attention_mask=sequence_masks, labels=batch_labels)
        loss, output = raw_output['loss'], raw_output['logits']
        logits = output.view(-1, model.num_labels)
        batch_predictions = torch.argmax(logits, dim=1)

        proper_labels = batch_labels.view(-1) != -100
        loss = loss_crt(logits, batch_labels.view(-1))

        filtered_labels = torch.masked_select(batch_labels.view(-1), proper_labels)
        filtered_predictions = torch.masked_select(batch_predictions, proper_labels)

        labels += filtered_labels.squeeze().tolist()
        predictions += filtered_predictions.tolist()

        batch_acc = accuracy_score(filtered_labels.cpu().numpy(), filtered_predictions.cpu().numpy())
        epoch_acc += batch_acc

        loss_scalar = loss.item()

        if idx % 500 == 0:
            print(epoch_acc/(idx + 1))
            print(batch_predictions)

        model.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss_scalar

    epoch_loss = epoch_loss/num_batches
    epoch_acc = epoch_acc/num_batches

    return epoch_loss, epoch_acc, labels

def eval_epoch(model, val_dataloader, loss_crt, device):
    model.eval()
    epoch_loss = 0.0
    epoch_acc = 0.0
    num_batches = len(val_dataloader)
    predictions = []
    labels = []
    with torch.no_grad():
        for idx, batch in tqdm(enumerate(val_dataloader)):
            batch_data, batch_labels = batch
            sequence_ids = batch_data['input_ids'].to(device, dtype=torch.long)
            sequence_masks = batch_data['attention_mask'].to(device)
            batch_labels = batch_labels.to(device)

            raw_output = model(input_ids=sequence_ids, attention_mask=sequence_masks, labels=batch_labels)
            loss, output = raw_output['loss'], raw_output['logits']
            logits = output.view(-1, model.num_labels)
            batch_predictions = torch.argmax(logits, dim=1)

            proper_labels = batch_labels.view(-1) != -100

            filtered_labels = torch.masked_select(batch_labels.view(-1), proper_labels)
            filtered_predictions = torch.masked_select(batch_predictions, proper_labels)

            labels += filtered_labels.squeeze().tolist()
            predictions += filtered_predictions.tolist()

            batch_acc = accuracy_score(filtered_labels.cpu().numpy(), filtered_predictions.cpu().numpy())
            epoch_acc += batch_acc

            loss_scalar = loss.item()

            epoch_loss += loss_scalar

    epoch_loss = epoch_loss/num_batches
    epoch_acc = epoch_acc/num_batches

    return epoch_loss, epoch_acc, labels

In [9]:
weights = np.zeros(16)
for sequence in y_train:
    for label in sequence:
        if label != -100:
            weights[label] += 1
max_weight = np.max(weights)
for index, weight in enumerate(weights):
    weights[index] = max_weight / weights[index]

print(weights)


[  1.          10.35858438  20.01783583  31.55568756  71.41922474
  80.65384615 112.59060403 585.92647059  57.16355811  18.93453725
 117.53097345 105.43962951 156.5540275   50.59428571 134.03868797
 120.69064748]


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# move the model to GPU (when available)
model.to(device)

# create a SGD optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)

# set up loss function
loss_criterion = nn.CrossEntropyLoss(weight=torch.as_tensor(weights, dtype=torch.float).to(device))

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
for epoch in range(1, EPOCHS+1):
    train_loss, train_accuracy, pepega_labels = train_epoch(model, train_dataloader, loss_criterion, optimizer, device)
    val_loss, val_accuracy, pepega_pepega_labels = eval_epoch(model, validation_dataloader, loss_criterion, device)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)
    print('\nEpoch %d'%(epoch))
    print('train loss: %10.8f, accuracy: %10.8f'%(train_loss, train_accuracy))
    print('val loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))


2it [00:00, 13.77it/s]

0.045454545454545456
tensor([ 2, 12, 13,  6,  5,  1, 15, 12,  8, 12,  5,  0, 10, 10, 10,  6, 10,  1,
         6, 10, 15,  0, 12,  6,  9, 11,  6, 12, 14, 12, 12,  6, 15,  6, 12, 11,
        11, 11,  5, 12, 11, 12, 12, 12, 12, 12, 12, 12, 12,  5,  1, 12,  5,  1,
         5,  5, 12, 12,  5,  5,  6,  5,  5,  5, 12, 12, 12, 11, 11, 12, 12,  1,
        11, 11, 12,  5, 12, 12, 12, 12, 12, 12, 12, 12,  5, 10, 12, 12,  5,  5,
        12, 12,  5,  5,  5, 12, 12,  6,  1, 12, 12,  1, 11, 15,  1,  1,  5, 12,
        15, 12, 12, 12, 12, 12, 12, 12, 12, 12,  5, 15, 12, 12,  1,  5,  1, 12,
        12, 12, 11, 12,  6, 15,  4, 15,  5,  2,  6,  1, 14, 14,  5, 14, 13,  1,
         1, 14,  4, 12, 12,  6,  6, 12, 12, 12,  1, 12, 12, 12, 12,  1, 12, 14,
        12, 15,  3, 11,  5,  1,  6,  6, 11, 15, 11,  1,  1,  1,  6,  1, 14, 10,
         4,  4, 15, 12, 12, 14,  6,  5, 11,  6,  8, 11,  1,  1,  5,  6, 15, 11,
        12,  5,  5,  5,  1,  1,  6,  5, 12,  1,  9,  6,  7,  5,  7,  5,  7, 11,
         4,  5,  3,

502it [00:39, 12.34it/s]

0.06711381425495123
tensor([10, 10, 10, 10, 10,  2, 10, 10, 10, 10, 10, 10, 10,  2, 10, 10, 10, 10,
         2, 10, 10, 10, 10, 10,  1, 10,  2, 10,  2, 10, 10, 10, 10,  2, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  2, 10,  2, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10,  2, 10, 10, 10, 10,  2, 10, 10, 10,
        10,  1, 10, 10, 10, 10, 10, 10, 10, 10,  1, 10, 10, 10,  2, 10, 10, 10,
        10, 10, 10,  2, 10, 10, 10, 10, 10, 10,  1, 10, 10,  2, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 13, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10,  1, 10, 10, 10,  1, 10, 10, 10, 10, 10, 10,
         2, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  1,  2, 10, 10, 10, 10,
         2, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  2, 10, 10, 10,
        10,  1, 10,  2, 10, 10,  2, 10,  1, 10, 10,  2,  2, 10, 10, 10, 10, 10,
        10,  1, 10, 10, 10, 10,  1, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 

1002it [01:19, 12.27it/s]

...

0.2589243668829135
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')


4502it [06:22, 12.27it/s]

0.2966039613055169
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1], device='cuda:0')


4932it [06:58, 11.79it/s]
1233it [00:17, 70.02it/s]


Epoch 10
train loss: 2.72471887, accuracy: 0.32516627
val loss: 2.40072243, accuracy: 0.07186798


In [ ]:
! nvidia-smi

In [11]:
model.save_pretrained("model_pretrained_SGD_128_2_10")
torch.save(model.state_dict(), "model_SGD_128_2_10.pt")

In [ ]:
# model.load_state_dict(torch.load("model_SGD_64_2.pt"))

In [ ]:
# model.from_pretrained("model_pretrained_SGD_64_2")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# move the model to GPU (when available)
model.to(device)

# create a SGD optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# set up loss function
loss_criterion = nn.CrossEntropyLoss()
val_loss, val_accuracy, pepega_pepega_labels = eval_epoch(model, validation_dataloader, loss_criterion, device)
print(val_accuracy)


In [12]:
def read_test(dataset, tokenizer):
    data = []
    max_length = 0
    counter = 0
    reshaped_dataset = []
    # reshaped_length = 4
    # for item in dataset:
    #     for i in range(0, len(item['tokens']), reshaped_length):
    #         reshaped_dataset.append(item['tokens'][i: min(i + reshaped_length, len(item['tokens']) ) ] )

    for item in dataset:
        reshaped_dataset.append(item['tokens'])

    for item in reshaped_dataset:
        # counter += len(item)
        prelucrate_item = []
        for token in item:
            prelucrate_item.append(re.sub(r"\W+", 'or', token))
        # print(prelucrate_item)
        sequence = tokenizer(
            prelucrate_item,
            is_split_into_words=True,
            padding="max_length",
            max_length=MAX_LENGTH,
            # truncation=True,
            return_offsets_mapping=True
        )
        # sequence = tokenizer.encode(
        #     prelucrate_item,
        #     is_pretokenized=True,
        # )
        sequence = {key: torch.as_tensor(value) for key, value in sequence.items()}
        data.append(sequence)

    #     if len(sequence['input_ids']) > max_length:
    #         print(tokenizer.convert_ids_to_tokens(sequence['input_ids']))
    #         print((sequence['offset_mapping']))
    #         print(item)
    #     max_length = max(len(sequence['input_ids']), max_length)
    # print(max_length)
    return data

reshaped_test_data = read_test(raw_test_data, tokenizer=tokenizer)

test_dataloader = DataLoader(
    dataset=reshaped_test_data,
    batch_size=BATCH_SIZE
)


In [13]:
def test_epoch(model, test_dataloader, device):
    model.eval()
    epoch_loss = 0.0
    num_batches = len(test_dataloader)
    predictions = []
    with torch.no_grad():
        for idx, batch in tqdm(enumerate(test_dataloader)):
            batch_data = batch
            sequence_ids = batch_data['input_ids'].to(device)
            sequence_masks = batch_data['attention_mask'].to(device)
            offset_mapping = batch_data['offset_mapping']

            raw_output = model(sequence_ids, attention_mask=sequence_masks)
            output =  raw_output['logits']
            logits = output.view(-1, model.num_labels)
            batch_predictions = torch.argmax(logits, dim=1)
            print(batch_predictions)


            filtered_predictions = []

            # raw_batch_predictions = torch.argmax(output, dim=2)
            # # print(offset_mapping.shape)
            # for index_bt, bt in enumerate(offset_mapping):
            #     for index, offset in enumerate(bt):
            #         if offset[0] == 0 and offset[1] != 0:
            #             filtered_predictions.append(raw_batch_predictions[index_bt][index])

            for index, offset in enumerate(offset_mapping.view(-1, 2)):
                if offset[0] == 0 and offset[1] != 0:
                    filtered_predictions.append(batch_predictions[index])

            predictions += filtered_predictions

    return predictions

all_predictions = test_epoch(model, test_dataloader, device)
print(len(all_predictions))

5it [00:00, 48.67it/s]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

10it [00:00, 48.42it/s]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')


15it [00:00, 48.81it/s]

...

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

1205it [00:23, 51.64it/s]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

1211it [00:23, 51.12it/s]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
73276


In [ ]:
print(all_predictions)

In [ ]:

print(re.sub(r'[\\\s+]', 't', '\\\\\\\nhahah    tester'))


